This script calucalates following similarity measures:
- BLEU
- ROUGE
- METEOR
- TER
- chrF and chrf++
- MoverScore
- BERT Score
- BLEURT Score
- Cosine Similarity

In [18]:
# !pip install -q rouge-score
# !pip install -q bert-score
# !pip install -q sacrebleu
# !pip install -q moverscore
# !pip install -q pyemd
# !pip install -q pytorch_pretrained_bert
# !pip install -q git+https://github.com/google-research/bleurt.git
# !pip install -q sentence_transformers

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
corpus = 'manual' # 'mrpc'/'manual'

folder_input_path = '/content/drive/My Drive/Colab Notebooks/5_Corpora/corpora/'
file_path = f'{corpus}-sentiment-triplet-corpus.csv'

In [21]:
import pandas as pd

df = pd.read_csv(folder_input_path + file_path)

# df = df.drop(['positive_cos_sim', 'negative_cos_sim'], axis=1)
df.head()

,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,positive_sentiment_label,positive_sentiment_score,negative_sentiment_label,negative_sentiment_score
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,POS,0.992210,NEG,0.874733
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,POS,0.986160,NEG,0.896712
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,POS,0.991108,NEG,0.982288
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,POS,0.992721,NEG,0.973961
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,POS,0.987900,NEG,0.977876


# BLEU

In [22]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def compute_bleu_for_triplet(original, positive_paraphrase, negative_paraphrase):
    # Tokenize the sentences
    original_tokens = original.split()
    positive_tokens = positive_paraphrase.split()
    negative_tokens = negative_paraphrase.split()

    # compute BLEU scores
    smoothie = SmoothingFunction().method4
    positive_bleu = sentence_bleu([original_tokens], positive_tokens, smoothing_function=smoothie)
    negative_bleu = sentence_bleu([original_tokens], negative_tokens, smoothing_function=smoothie)

    return positive_bleu, negative_bleu

In [23]:
%%time

# Add new columns for scores of positive and negative paraphrases
for index, row in df.iterrows():
    positive_scores, negative_scores = compute_bleu_for_triplet(row['original'], row['positive'], row['negative'])

    # Assigning scores to the DataFrame
    df.at[index, 'positive_bleu'] = positive_scores
    df.at[index, 'negative_bleu'] = negative_scores

df.head()

CPU times: user 73 ms, sys: 0 ns, total: 73 ms
Wall time: 73 ms


,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,positive_sentiment_label,positive_sentiment_score,negative_sentiment_label,negative_sentiment_score,positive_bleu,negative_bleu
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,POS,0.992210,NEG,0.874733,0.036918,0.071031
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,POS,0.986160,NEG,0.896712,0.000000,0.025733
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,POS,0.991108,NEG,0.982288,0.097177,0.121211
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,POS,0.992721,NEG,0.973961,0.034011,0.034011
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,POS,0.987900,NEG,0.977876,0.084704,0.023457


# ROUGE

In [24]:
from rouge_score import rouge_scorer

def compute_rouge_f1_scores(original, paraphrase):
    # Create a ROUGE scorer instance
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # compute ROUGE scores for the paraphrase
    scores = scorer.score(original, paraphrase)

    # Extract and return the F1-scores for ROUGE-1, ROUGE-2, and ROUGE-L
    rouge1_f1 = scores['rouge1'].fmeasure
    rouge2_f1 = scores['rouge2'].fmeasure
    rougeL_f1 = scores['rougeL'].fmeasure

    return rouge1_f1, rouge2_f1, rougeL_f1

In [25]:
%%time

# Add new columns for F1 scores of positive and negative paraphrases
for index, row in df.iterrows():
    positive_scores = compute_rouge_f1_scores(row['original'], row['positive'])
    negative_scores = compute_rouge_f1_scores(row['original'], row['negative'])

    # Assigning F1-scores to the DataFrame
    df.at[index, 'positive_rouge_n'] = positive_scores[0]
    df.at[index, 'positive_rouge_s'] = positive_scores[1]
    df.at[index, 'positive_rouge_l'] = positive_scores[2]

    df.at[index, 'negative_rouge_n'] = negative_scores[0]
    df.at[index, 'negative_rouge_s'] = negative_scores[1]
    df.at[index, 'negative_rouge_l'] = negative_scores[2]

df.head()

CPU times: user 173 ms, sys: 911 µs, total: 174 ms
Wall time: 175 ms


,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,negative_sentiment_label,negative_sentiment_score,positive_bleu,negative_bleu,positive_rouge_n,positive_rouge_s,positive_rouge_l,negative_rouge_n,negative_rouge_s,negative_rouge_l
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,NEG,0.874733,0.036918,0.071031,0.533333,0.000000,0.400000,0.400000,0.307692,0.400000
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,NEG,0.896712,0.000000,0.025733,0.266667,0.000000,0.133333,0.352941,0.000000,0.117647
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,NEG,0.982288,0.097177,0.121211,0.545455,0.222222,0.545455,0.545455,0.222222,0.363636
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,NEG,0.973961,0.034011,0.034011,0.333333,0.000000,0.333333,0.500000,0.000000,0.333333
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,NEG,0.977876,0.084704,0.023457,0.533333,0.153846,0.533333,0.375000,0.142857,0.250000


# METEOR

In [26]:
!python -m nltk.downloader punkt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
%%time

from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import nltk

nltk.download('wordnet')

def compute_meteor(original, paraphrase):
    # Tokenizing the sentences
    original_tokens = word_tokenize(original)
    paraphrase_tokens = word_tokenize(paraphrase)

    # Computing METEOR score
    score = meteor_score([original_tokens], paraphrase_tokens)

    return score

# Add new columns for scores of positive and negative paraphrases
for index, row in df.iterrows():
    # Assigning scores to the DataFrame
    df.at[index, 'positive_meteor'] = compute_meteor(row['original'], row['positive'])
    df.at[index, 'negative_meteor'] = compute_meteor(row['original'], row['negative'])

df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CPU times: user 366 ms, sys: 4.5 ms, total: 370 ms
Wall time: 375 ms


,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,positive_bleu,negative_bleu,positive_rouge_n,positive_rouge_s,positive_rouge_l,negative_rouge_n,negative_rouge_s,negative_rouge_l,positive_meteor,negative_meteor
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,0.036918,0.071031,0.533333,0.000000,0.400000,0.400000,0.307692,0.400000,0.417978,0.421348
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,0.000000,0.025733,0.266667,0.000000,0.133333,0.352941,0.000000,0.117647,0.168539,0.219780
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,0.097177,0.121211,0.545455,0.222222,0.545455,0.545455,0.222222,0.363636,0.517418,0.517418
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,0.034011,0.034011,0.333333,0.000000,0.333333,0.500000,0.000000,0.333333,0.277778,0.370370
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,0.084704,0.023457,0.533333,0.153846,0.533333,0.375000,0.142857,0.250000,0.619444,0.426520


# TER and chrF/chrf++

In [28]:
%%time

import sacrebleu

def compute_ter(original, paraphrase):
    ter = sacrebleu.metrics.TER()
    score = ter.sentence_score(paraphrase, [original])
    return score.score

def compute_chrf(original, paraphrase, beta=2, word_order=False):
    # Compute chrF and optionally chrF++ if word_order is True
    chrf_score = sacrebleu.corpus_chrf(paraphrase, [original], beta=beta, word_order=word_order).score

    return chrf_score

# Add new columns for scores of positive and negative paraphrases
for index, row in df.iterrows():
    # Assigning scores to the DataFrame
    df.at[index, 'positive_ter'] = compute_ter(row['original'], row['positive'])
    df.at[index, 'negative_ter'] = compute_ter(row['original'], row['negative'])

    df.at[index, 'positive_chrf'] = compute_chrf(row['original'], row['positive'])
    df.at[index, 'negative_chrf'] = compute_chrf(row['original'], row['negative'])

    df.at[index, 'positive_chrfpp'] = compute_chrf(row['original'], row['positive'], word_order=True)
    df.at[index, 'negative_chrfpp'] = compute_chrf(row['original'], row['negative'], word_order=True)

df.head()

CPU times: user 1.22 s, sys: 5.18 ms, total: 1.23 s
Wall time: 1.23 s


,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,negative_rouge_s,negative_rouge_l,positive_meteor,negative_meteor,positive_ter,negative_ter,positive_chrf,negative_chrf,positive_chrfpp,negative_chrfpp
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,0.307692,0.400000,0.417978,0.421348,75.000000,87.500000,28.846154,6.451613,28.846154,6.451613
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,0.000000,0.117647,0.168539,0.219780,100.000000,112.500000,9.677419,9.803922,9.677419,9.803922
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,0.222222,0.363636,0.517418,0.517418,80.000000,60.000000,32.894737,9.933775,32.894737,9.933775
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,0.000000,0.333333,0.277778,0.370370,150.000000,150.000000,16.949153,17.094017,16.949153,17.094017
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,0.142857,0.250000,0.619444,0.426520,66.666667,133.333333,36.184211,10.135135,36.184211,10.135135


# MoverScore

In [29]:
%%time

import os
from moverscore import get_idf_dict, word_mover_score
os.environ['MOVERSCORE_MODEL'] = "albert-base-v2"

def compute_moverscore(original, paraphrase):
    # Calculate IDF values based on both the reference and system sentences
    idf_dict_original = get_idf_dict([original])  # IDF for references
    idf_dict_paraphrase = get_idf_dict([paraphrase])   # IDF for system outputs

    # Compute MoverScore
    scores = word_mover_score([original], [paraphrase], idf_dict_original, idf_dict_paraphrase, stop_words=[], n_gram=1, remove_subwords=True)

    # Return the first (and only) score since we have one-to-one sentence comparison
    return scores[0]


for index, row in df.iterrows():
    df.at[index, 'positive_mover'] = compute_moverscore(row['original'], row['positive'])
    df.at[index, 'negative_mover'] = compute_moverscore(row['original'], row['negative'])

df.head()

CPU times: user 26.3 s, sys: 1min 36s, total: 2min 2s
Wall time: 3min 53s


,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,positive_meteor,negative_meteor,positive_ter,negative_ter,positive_chrf,negative_chrf,positive_chrfpp,negative_chrfpp,positive_mover,negative_mover
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,0.417978,0.421348,75.000000,87.500000,28.846154,6.451613,28.846154,6.451613,1.0,1.0
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,0.168539,0.219780,100.000000,112.500000,9.677419,9.803922,9.677419,9.803922,1.0,1.0
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,0.517418,0.517418,80.000000,60.000000,32.894737,9.933775,32.894737,9.933775,1.0,1.0
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,0.277778,0.370370,150.000000,150.000000,16.949153,17.094017,16.949153,17.094017,1.0,1.0
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,0.619444,0.426520,66.666667,133.333333,36.184211,10.135135,36.184211,10.135135,1.0,1.0


# BERT Score
https://haticeozbolat17.medium.com/text-summarization-how-to-calculate-bertscore-771a51022964

In [30]:
%%time

from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer

scorer = BERTScorer(model_type='bert-base-uncased', device=None, lang='en')


def compute_bertscore(original, paraphrase):
    # Since BERTScore expects lists of sentences, wrap inputs in lists
    P, R, F1 = scorer.score([paraphrase], [original], verbose=False)

    # Return the F1 score
    return F1.item()

# Add new columns for scores of positive and negative paraphrases
for index, row in df.iterrows():

    # Assigning scores to the DataFrame
    df.at[index, 'positive_bert'] = compute_bertscore(row['original'], row['positive'])
    df.at[index, 'negative_bert'] = compute_bertscore(row['original'], row['negative'])

df.head()

CPU times: user 5.75 s, sys: 267 ms, total: 6.02 s
Wall time: 6.14 s


,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,positive_ter,negative_ter,positive_chrf,negative_chrf,positive_chrfpp,negative_chrfpp,positive_mover,negative_mover,positive_bert,negative_bert
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,75.000000,87.500000,28.846154,6.451613,28.846154,6.451613,1.0,1.0,0.565429,0.628388
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,100.000000,112.500000,9.677419,9.803922,9.677419,9.803922,1.0,1.0,0.600216,0.546758
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,80.000000,60.000000,32.894737,9.933775,32.894737,9.933775,1.0,1.0,0.786924,0.793672
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,150.000000,150.000000,16.949153,17.094017,16.949153,17.094017,1.0,1.0,0.611858,0.617260
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,66.666667,133.333333,36.184211,10.135135,36.184211,10.135135,1.0,1.0,0.702324,0.683737


In [31]:
# from bert_score import score

# def compute_bertscore(original, paraphrase):
#     # Since BERTScore expects lists of sentences, wrap inputs in lists
#     P, R, F1 = score([paraphrase], [original], lang="en", verbose=False)

#     # Return the F1 score
#     return F1.item()

# # Add new columns for scores of positive and negative paraphrases
# for index, row in df.iterrows():

#     # Assigning scores to the DataFrame
#     df.at[index, 'positive_bert'] = compute_bertscore(row['original'], row['positive'])
#     df.at[index, 'negative_bert'] = compute_bertscore(row['original'], row['negative'])

# df.head()

# BLEURT Score

In [32]:
from bleurt import score

# Load the BLEURT scorer with the path to your downloaded model
bleurt_scorer = score.BleurtScorer("/content/drive/My Drive/Colab Notebooks/5_Corpora/bleurt-model")

def compute_bleurt_score(original, paraphrase):
    scores = bleurt_scorer.score(references=[original], candidates=[paraphrase])
    return scores[0]

# Add new columns for scores of positive and negative paraphrases
for index, row in df.iterrows():

    # Assigning scores to the DataFrame
    df.at[index, 'positive_bleurt'] = compute_bleurt_score(row['original'], row['positive'])
    df.at[index, 'negative_bleurt'] = compute_bleurt_score(row['original'], row['negative'])

df.head()

,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,positive_chrf,negative_chrf,positive_chrfpp,negative_chrfpp,positive_mover,negative_mover,positive_bert,negative_bert,positive_bleurt,negative_bleurt
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,28.846154,6.451613,28.846154,6.451613,1.0,1.0,0.565429,0.628388,0.578367,0.528240
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,9.677419,9.803922,9.677419,9.803922,1.0,1.0,0.600216,0.546758,0.450409,0.464059
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,32.894737,9.933775,32.894737,9.933775,1.0,1.0,0.786924,0.793672,0.717583,0.692635
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,16.949153,17.094017,16.949153,17.094017,1.0,1.0,0.611858,0.617260,0.508035,0.570751
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,36.184211,10.135135,36.184211,10.135135,1.0,1.0,0.702324,0.683737,0.624876,0.512334


# Cosine Similarity

In [33]:
from sentence_transformers import SentenceTransformer, util

# Load the model
model = SentenceTransformer('all-mpnet-base-v2')

def compute_cosine_similarity(original, paraphrase, threshold=0.75):
    # Compute embeddings
    embeddings_original = model.encode(original, convert_to_tensor=True)
    embeddings_paraphrase = model.encode(paraphrase, convert_to_tensor=True)

    # Compute cosine similarity
    cosine_scores = util.pytorch_cos_sim(embeddings_original, embeddings_paraphrase)

    return cosine_scores.item() # >= threshold

for index, row in df.iterrows():
    df.at[index, 'positive_cosine_similarity'] = compute_cosine_similarity(row['original'], row['positive'])
    df.at[index, 'negative_cosine_similarity'] = compute_cosine_similarity(row['original'], row['negative'])

df.head()

,original,positive,negative,positive_cos_sim,negative_cos_sim,original_label,positive_label,negative_label,original_sentiment_label,original_sentiment_score,...,positive_chrfpp,negative_chrfpp,positive_mover,negative_mover,positive_bert,negative_bert,positive_bleurt,negative_bleurt,positive_cosine_similarity,negative_cosine_similarity
0,I usually wake up early in the morning.,I enjoy the freshness of early mornings.,Waking up early often feels too abrupt.,0.615547,0.768586,NEU,POS,NEG,NEU,0.945723,...,28.846154,6.451613,1.0,1.0,0.565429,0.628388,0.578367,0.528240,0.615547,0.768586
1,I regularly take a walk in the evening.,My evening walks are refreshing and peaceful.,Evening walks can be tiring after a long day.,0.759722,0.810249,NEU,POS,NEG,NEU,0.777473,...,9.677419,9.803922,1.0,1.0,0.600216,0.546758,0.450409,0.464059,0.759722,0.810249
2,The weather is unpredictable today.,The weather today holds exciting surprises.,Today's weather is frustratingly inconsistent.,0.769934,0.762804,NEU,POS,NEG,NEU,0.939865,...,32.894737,9.933775,1.0,1.0,0.786924,0.793672,0.717583,0.692635,0.769934,0.762804
3,The restaurant offers food.,The cuisine at this restaurant is delightfully...,The food at this restaurant is barely passable.,0.693150,0.575224,NEU,POS,NEG,NEU,0.651773,...,16.949153,17.094017,1.0,1.0,0.611858,0.617260,0.508035,0.570751,0.682496,0.560049
4,This movie features a standard plot.,This movie boasts a captivating and thought-pr...,The plot of this movie is confusing and poorly...,0.594192,0.581412,NEU,POS,NEG,NEU,0.928657,...,36.184211,10.135135,1.0,1.0,0.702324,0.683737,0.624876,0.512334,0.566464,0.548903


In [34]:
df.to_csv(folder_input_path + f'{corpus}-triplet-corpus.csv', index=False)